In [1]:
import os

!pip install tensorflow
!pip install tensorflow==1.15
!pip install tensorflow-gpu==1.15

os.system("export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/extras/CUPTI/lib64")

ERROR: Could not find a version that satisfies the requirement tensorflow==1.15 (from versions: 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.7.0rc0, 2.7.0rc1, 2.7.0)
ERROR: No matching distribution found for tensorflow==1.15
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.15 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.6.0, 2.6.1, 2.6.2, 2.7.0rc0, 2.7.0rc1, 2.7.0)
ERROR: No matching distribution found for tensorflow-gpu==1.15


0

In [2]:
!pip3 install --upgrade tensorflow-gpu

     |████████████████████████████████| 489.7 MB 18 kB/s              


In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import os
import csv
import numpy as np
import pandas as pd
import rasterio
import time

wildfire = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles'
lcp = '/home/qweddww123/farsite/TrainDataGen/GetLCPdata/LCP/lcp_extract'

input_data_size = 15
cnt = 0
for year in sorted(os.listdir(wildfire), key=int):
    for target_fire in sorted(os.listdir(wildfire+'/'+year), key=str):
        path = wildfire+'/'+year
        if(target_fire[0] == '.'):
            continue
        elif(target_fire.split('_')[0] == 'ak' or target_fire.split('_')[0] == 'AK'):
            continue
        elif(target_fire.split('_')[0] == 'hi' or target_fire.split('_')[0] == 'ca'):
            continue
        else:
            cnt+=1

print(cnt)
train_images = np.zeros((cnt*int(600/15*600/15),15 ,15, 15))
train_labels = np.zeros(cnt*int(600/15*600/15))
print(train_images.shape)
print(train_labels.shape)
index = 0
for year in sorted(os.listdir(wildfire), key=int):
    print(year, '***************************************************************************************************************')
    for target_fire in sorted(os.listdir(wildfire+'/'+year), key=str):
        path = wildfire+'/'+year
        if(target_fire[0] == '.'):
            continue
        elif(target_fire.split('_')[0] == 'ak' or target_fire.split('_')[0] == 'AK'):
            continue
        elif(target_fire.split('_')[0] == 'hi' or target_fire.split('_')[0] == 'ca'):
            continue
        else:
            #print(target_fire)
            dirpath = sorted(os.listdir(wildfire+'/'+year+'/'+target_fire), key=str)[0]

            try:
                with open(wildfire+'/'+year+'/'+target_fire+'/'+dirpath+'/MLinput/'+'input_img.csv', 'r') as f:
                    reader = csv.reader(f)
                    examples = list(reader)

                    #read semantics csv file
                    input_img = []
                    for row in examples:
                        nwrow = []
                        for r in row:
                            nwrow.append(eval(r))
                        input_img.append(nwrow)
                    input_img = np.array(input_img)

                    input_img = input_img.astype(np.float64)

                with open(wildfire+'/'+year+'/'+target_fire+'/'+dirpath+"/output.csv", 'r') as c:
                    reader1 = csv.reader(c)
                    comp_lines = list(reader1)
                    
                    #read computing load csv file
                    output_img = []
                    for row in comp_lines:
                        output_img.append(row)
                    output_img = np.array(output_img)
            except:
                continue

            output_img = np.array(output_img, dtype=np.float64)
            #print(output_img.dtype)
            #clip the input image matrix into 15*15*15*1 sized matrix
            #train_img.shape == 15*15*15*1
            #(length of raster data - input_shape x) + 1 = number of iteration
            #print(raster.height/input_data_size)
            #print(int(raster.height/input_data_size))
            start = time.time()
            num = 0
            
            for y in range(int(600/15)):
                for x in range(int(600/15)):
                    for i in range(15):
                        for j in range(15):
                            train_labels[index] += round(output_img[i+y*15][j+x*15], 3)
                            for k in range(15):
                                train_images[index][i][j][k] = input_img[i][j+y*15][k+x*15]
                    index += 1

            #for i in range(1600):
            #    print(round(train_labels[i], 3), end=" ")

            #print()
            #time.sleep(10)
            #print("time: ", time.time() - start)

model = models.Sequential()
model.add(layers.Conv3D(filters=128, kernel_size=3, activation="relu", input_shape=(input_data_size, input_data_size, input_data_size, 1)))
#model.add(layers.MaxPooling3D(pool_size=2))
model.add(layers.Conv3D(filters=64, kernel_size=3, activation="relu"))
#model.add(layers.MaxPooling3D(pool_size=2))
model.add(layers.Conv3D(filters=64, kernel_size=3, activation="relu"))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5)

2021-11-10 21:09:47.553733: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/melodic/lib:/home/qweddww123/OpenDDS-3.14/ACE_wrappers/lib:/home/qweddww123/OpenDDS-3.14/lib
2021-11-10 21:09:47.553801: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


539
(862400, 15, 15, 15)
(862400,)
2016 ***************************************************************************************************************
2019 ***************************************************************************************************************


2021-11-10 22:26:23.118131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/melodic/lib:/home/qweddww123/OpenDDS-3.14/ACE_wrappers/lib:/home/qweddww123/OpenDDS-3.14/lib
2021-11-10 22:26:23.118261: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/melodic/lib:/home/qweddww123/OpenDDS-3.14/ACE_wrappers/lib:/home/qweddww123/OpenDDS-3.14/lib
2021-11-10 22:26:23.118360: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/melodic/lib:/home/qweddww123/OpenDDS-3.14/ACE_w

In [ ]:

df = pd.DataFrame(input_img)
df.to_csv(wildfire+'/'+year+'/'+target_fire+'/'+shpdir+'/'+"MLinput/Number_of_points.csv", header=False, index=False)

In [ ]:
import os
os.system("lspci | grep -i nvidia")

01:00.0 VGA compatible controller: NVIDIA Corporation Device 2206 (rev a1)
01:00.1 Audio device: NVIDIA Corporation Device 1aef (rev a1)


0